In [ ]:
class MutationLibrary:
    """突变库：定义各种工程损伤的数学模型"""
    
    def __init__(self, beam_length=5.0):
        self.beam_length = beam_length
        self.mutation_types = self._initialize_mutation_types()
    
    def _initialize_mutation_types(self):
        """初始化所有突变类型"""
        return {
            #固定端
                'joint_misaligned': {
                'category': '连接界面',
                'name': '错位连接',
                'description': '刚度波动，先升后降',
                'default_params': {
                    'amplitude_range': [(-0.15, -0.05), (0.05, 0.15)],  # 两个振幅范围
                    'width_range': (0.2, 0.3),
                    'shape': 'bimodal',  # 双峰分布
                    'symmetry': 'asymmetric',
                    'position_constraint': 'any',
                    'probability_weight': 0.1,
                },
                'generate_function': self._generate_joint_misaligned
            },
            # ============ 1. 连接界面突变 ============
            'joint_normal': {
                'category': '连接界面',
                'name': '正常连接界面',
                'description': '轻微刚度过渡，±10%变化',
                'default_params': {
                    'amplitude_range': (-0.1, 0.1),  # 刚度变化率
                    'width_range': (0.04, 0.06),     # 影响宽度 (m)
                    'shape': 'gaussian',
                    'symmetry': 'symmetric',
                    'position_constraint': 'any',  # 可以在任何位置
                    'probability_weight': 0.3,     # 出现概率权重
                },
                'generate_function': self._generate_joint_normal
            },
            
            'joint_loose': {
                'category': '连接界面',
                'name': '松动连接',
                'description': '刚度显著下降，20-40%损失，左侧影响更大',
                'default_params': {
                    'amplitude_range': (-0.4, -0.2),
                    'width_range': (0.10, 0.25),
                    'shape': 'asymmetric_gaussian',  # 特殊形状
                    'symmetry': 'left_biased',  # 左侧影响大
                    'position_constraint': 'any',
                    'probability_weight': 0.25,
                },
                'generate_function': self._generate_joint_loose
            },
            
            'joint_over_tight': {
                'category': '连接界面',
                'name': '过紧连接',
                'description': '刚度异常上升，20-30%增加',
                'default_params': {
                    'amplitude_range': (0.2, 0.3),
                    'width_range': (0.10, 0.25),
                    'shape': 'sharp_gaussian',  # 陡峭变化
                    'symmetry': 'symmetric',
                    'position_constraint': 'any',
                    'probability_weight': 0.15,
                },
                'generate_function': self._generate_joint_over_tight
            },
            
            'joint_misaligned': {
                'category': '连接界面',
                'name': '错位连接',
                'description': '刚度波动，先升后降',
                'default_params': {
                    'amplitude_range': (-0.15, 0.15),  # 正负都有
                    'width_range': (0.2, 0.3),
                    'shape': 'bimodal',  # 双峰分布
                    'symmetry': 'asymmetric',
                    'position_constraint': 'any',
                    'probability_weight': 0.1,
                },
                'generate_function': self._generate_joint_misaligned
            },
            
            # ============ 2. 外部损伤突变 ============
            'overload_damage': {
                'category': '外部损伤',
                'name': '过载损伤',
                'description': '屈服后刚度下降，大范围',
                'default_params': {
                    'amplitude_range': (-0.25, -0.15),
                    'width_range': (0.3, 1.0),
                    'shape': 'plateau',  # 平台状
                    'symmetry': 'symmetric',
                    'position_constraint': 'middle',  # 通常在中间区域
                    'probability_weight': 0.2,
                },
                'generate_function': self._generate_overload_damage
            },
            
            'scratch_damage': {
                'category': '外部损伤',
                'name': '划痕/刮伤',
                'description': '线状表面损伤，浅层影响',
                'default_params': {
                    'amplitude_range': (-0.2, -0.1),
                    'width_range': (0.005, 0.01),  # 很窄
                    'length_range': (0.1, 0.8),     # 但长度可能很长
                    'shape': 'linear_decay',
                    'symmetry': 'unidirectional',
                    'position_constraint': 'any',
                    'probability_weight': 0.15,
                },
                'generate_function': self._generate_scratch_damage
            },
            
            'impact_dent': {
                'category': '外部损伤',
                'name': '冲击凹陷',
                'description': '局部凹陷，中心下降，边缘波动',
                'default_params': {
                    'amplitude_range': (-0.5, -0.2),
                    'diameter_range': (0.05, 0.15),  # 直径
                    'shape': 'impact_wave',  # 冲击波形
                    'symmetry': 'radial',
                    'position_constraint': 'any',
                    'probability_weight': 0.1,
                },
                'generate_function': self._generate_impact_dent
            },
            
            # ============ 3. 腐蚀/磨损突变 ============
            'uniform_corrosion': {
                'category': '腐蚀磨损',
                'name': '均匀腐蚀',
                'description': '大面积均匀刚度下降',
                'default_params': {
                    'amplitude_range': (-0.3, -0.1),
                    'width_range': (0.5, 2.0),
                    'shape': 'uniform',
                    'symmetry': 'symmetric',
                    'position_constraint': 'any',
                    'probability_weight': 0.1,
                },
                'generate_function': self._generate_uniform_corrosion
            },
            
            # ============ 4. 裂缝损伤突变 ============
            'hairline_crack': {
                'category': '裂缝损伤',
                'name': '发丝裂缝',
                'description': '极窄区域，轻微刚度下降',
                'default_params': {
                    'amplitude_range': (-0.15, -0.05),
                    'width_range': (0.002, 0.005),  # 2-5mm
                    'shape': 'sharp_step',  # 陡峭不连续
                    'symmetry': 'symmetric',
                    'position_constraint': 'stress_concentration',  # 应力集中区
                    'probability_weight': 0.08,
                },
                'generate_function': self._generate_hairline_crack
            },
            
            'surface_crack': {
                'category': '裂缝损伤',
                'name': '表面裂缝',
                'description': '一侧开放，渐进式下降',
                'default_params': {
                    'amplitude_range': (-0.2, -0.1),
                    'width_range': (0.01, 0.03),
                    'shape': 'exponential_decay',
                    'symmetry': 'one_sided',  # 单侧
                    'position_constraint': 'surface',
                    'probability_weight': 0.07,
                },
                'generate_function': self._generate_surface_crack
            },
            
            'fatigue_crack_cluster': {
                'category': '裂缝损伤',
                'name': '疲劳裂纹群',
                'description': '多个小裂纹聚集，通常在根部',
                'default_params': {
                    'amplitude_range': (-0.3, -0.2),  # 整体
                    'width_range': (0.1, 0.3),        # 整体宽度
                    'num_cracks_range': (3, 7),       # 裂纹数量
                    'individual_amplitude_range': (-0.1, -0.05),  # 单个裂纹
                    'shape': 'cluster',
                    'symmetry': 'clustered',
                    'position_constraint': 'root_only',  # 只在根部
                    'probability_weight': 0.05,
                },
                'generate_function': self._generate_fatigue_crack_cluster
            }
        }
